In [1]:
import re
from nltk import Tree

def parser(expression):
    old = expression.replace(' ', '_').replace('>_(', '> (').replace(')_(', ') (').replace(')_)', ') )').replace(')_)', ') )').replace(' (', '(')
    new = ''
    flag = False
    for x in range(0, len(old) - 1):        
        if old[x] == '<':
            flag = True
        if old[x] == '>':
            flag = False
            
        if flag == True:
            if old[x] == '(':
                new += '{'
            elif old[x] == ')':
                new += '}'
            else:
                new += old[x]
        else:
            new += old[x]
    new += old[len(old)-1]
    return new

In [2]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append(pos['pos'])
    return res

In [3]:
def insert_pos_tag(exp, pos):
    count = 0
    res = ''
    for x in range(0, len(exp)):
        if exp[x] == 'S' and exp[x+1]==' ' and exp[x+2] == 'P':
            res += 'S '
            res += pos[count]
            count += 1
            x += 4
        else:
            res += exp[x]
    return res

In [4]:
def direction(exp):
    cont = False
    for x in exp:
        if x == '{':
            cont = True
        elif x == '}':
            cont = False
            continue
        if cont == True:
            continue
        if x == '/':
            return '/'
        elif x == '\\':
            return '\\'
    return False

In [5]:
def is_type_raising(tree):
    tree_string = str(tree)
    
    # check type raising
    exp = tree_string.split('_')[1]
    pattern_1 = r'(.*?)\\(.*?){(.*?)/(.*?)}'
    pattern_2 = r'(.*?)/(.*?){(.*?)\\(.*?)}'
    
    match = False
    if re.search(pattern_1, exp):
        match = True
    elif re.search(pattern_2, exp):
        match = True
        
    sub = []
    for subtree in tree:
        sub.append(subtree)
    if len(sub) == 1 and match:
        return True
    else:
        return False

In [6]:
def score(tree):
    tree_string = str(tree)
#     print(tree_string)
    if tree_string[2] == 'L':
        pos = tree_string.split('_')[3]
        word = tree_string.split('_')[5]
        
        #                        #
        # masukin rulenya disini #
        #                        #
        if 'JJ' in pos:
            return '10_'
    else:
        return False

In [7]:
import nltk
from nltk.sem.logic import *

read_expr = nltk.sem.Expression.fromstring

def lambda_calculus(tree):
    tree_string = str(tree)
    
    if tree_string[2] == 'L':
        pos = tree_string.split('_')[3]
        word = tree_string.split('_')[5]
        
        #                        #
        # masukin rulenya disini #
        #                        #
        r_word = ['PRP', 'FW', 'NN']
        if pos in r_word:
            return read_expr(r'(' + word + ')')
#         elif 'JJ' in pos:
#             return read_expr(r'(10)')
        elif 'VB' in pos:
            if '{S[dcl]\\NP}/{S[adj]\\NP}' in tree_string.split('_')[1]:
                return read_expr(r'\X.(X)')
            else:
                return read_expr(r'\X.' + word + '(X)')
        else:
            return read_expr(r'\X.(X)')
    
    # ini cuma masukin ke array 
    chunk = []
    sub = []
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            sub.append(subtree)
            
            # chunk temp array
            subtree_str_array = str(subtree).split('_')
            if subtree_str_array[0][2] == 'L':
                if subtree_str_array[3] == 'NN':
                    chunk.append(subtree_str_array[5])
    
    
    # chunk noun phrase
    if len(chunk) == 2:
        chunk_str = '_'.join(chunk)
        return read_expr(r'(' + chunk_str + ')')
    
    # error anak 1
    if len(sub) == 1:            
        return lambda_calculus(sub[0])
                        
    # urutan operasi lambda calculusnya    
    first = sub[0]
    second = sub[1]
    
    if is_type_raising(first):
        if direction(str(first).split('_')[1]) == '/':
            x = read_expr(r'\F x.F(x, ' + str(lambda_calculus(first)) + ')')
            y = lambda_calculus(second)
            return ApplicationExpression(x, y).simplify()
    
    if is_type_raising(second):
        if direction(str(second).split('_')[1]) == '/':
            x = lambda_calculus(first)
            y = read_expr(r'\F x.F(x, ' + str(lambda_calculus(second)) + ')')
            return ApplicationExpression(x, y).simplify()
        
    if score(second):
        x = str(lambda_calculus(first))
        if x == '\X.X':
            return read_expr(r'\X.' + score(second) + '(X)')
        new = ''
        for a in x:
            new += a
            if a == '.':
                new += score(second)
        return read_expr(r'' + new)
        
        
    length_1 = len(str(sub[0]).split('_')[1].replace('\\', '/').split('/'))
    length_2 = len(str(sub[1]).split('_')[1].replace('\\', '/').split('/'))
    if length_2 > length_1:
        first = sub[1]
        second = sub[0]
            
    
    # rekursi
    return ApplicationExpression(lambda_calculus(first), lambda_calculus(second)).simplify()

In [8]:
from_res = '(<T S[dcl] 0 2> (<T NP 0 2> (<L NP[nb]/N POS POS the NP[nb]/N>) (<L N POS POS hotel N>) ) (<T S[dcl]\\NP 0 2> (<L (S[dcl]\\NP)/NP POS POS had (S[dcl]\\NP)/NP>) (<T NP 0 2> (<L NP[nb]/N POS POS an NP[nb]/N>) (<T N 0 2> (<L N/N POS POS exceptional N/N>) (<L N POS POS service N>) ) ) ) )'
pos_tagged = insert_pos_tag(from_res, pos_tag('the hotel had a exceptional service'))

hasil = parser(pos_tagged)

from nltk import Tree
tree = Tree.fromstring(hasil)
# t.pretty_print()

In [9]:
lambda_calculus(tree)

<ApplicationExpression had(service,hotel)>

In [10]:
# hasil

In [11]:
# pos_tagged

In [12]:
# print(str(tree).replace('_', ' '))

In [13]:
import requests

def glue_process(sent):
    url = "http://localhost:5000/ccgParsing"
    data = {"sent": sent}
    r = requests.post(url, data=data)

    res = r.json()
    
    from_res = res['tree']
    pos_tagged = insert_pos_tag(from_res, pos_tag(data['sent']))

    hasil = parser(pos_tagged)

    tree = Tree.fromstring(hasil)
#     print(str(tree).replace('_', ' '))
    return lambda_calculus(tree)

In [14]:
glue_process('I bought my canon g3 about a month ago and i have to say i am very satisfied')
# glue_process('the hotel had an exceptional service')

<ApplicationExpression have(say(10_(i)),i,month(bought(canon_g3),I))>

In [15]:
glue_process('the breakfast that the restaurant served daily was excellent')

<ApplicationExpression 10_(10_served(breakfast,restaurant))>

In [16]:
#have(say(satisfied_10_(i)),i) & bought(canon_g3,I)

In [17]:
glue_process('just received this camera two days ago and already love the features it has . ')

<ApplicationExpression love(has(\X.X,it),\X.X,received(camera),\X.X,\X.X)>

In [18]:
a = read_expr(r'(10_(10_served(breakfast,restaurant)))')
b = read_expr(r'\X.10_(X)')
ApplicationExpression(b, a).simplify()

<ApplicationExpression 10_(10_(10_served(breakfast,restaurant)))>

In [19]:
x = read_expr(r'\X.10X')
restaurant = read_expr(r'restaurant')

second = read_expr(r'\X.serve(X)')
third = second(x, restaurant).simplify()
z = read_expr(r'X')
x

<LambdaExpression \X.10X>

In [20]:
y = read_expr(r'\X.10(X)')
x = read_expr(r'breakfast')
ApplicationExpression(y,x).simplify()

<ApplicationExpression 10(breakfast)>

In [21]:
zero = read_expr(r'\F x.F(x, ' + 'res' + ')')

one = read_expr(r'\x.\y.10serve(x, y)')
third = zero(one).simplify()
print(zero)
print(one)
print(third)
ApplicationExpression(third, read_expr(r'breakfast')).simplify()

\F x.F(x,res)
\x y.10serve(x,y)
\x.10serve(x,res)


<ApplicationExpression 10serve(breakfast,res)>

In [22]:
import pandas as pd
# input file
df = pd.read_csv("dataset.csv")
# preprocess
semkal = df['review']
labels = df["target"]

In [23]:
for x in range(0, 100):
    try:
        print(glue_process(semkal[x]), labels[x])
    except:
        continue

satisfied(purchase,\X.X,purchased(canon(powershot_g3)),\X.X,\X.X) 1
week(\x.was(asked(take(vacationing(group,picture))),x,i),trip,fact,use(\X.X),camera,\X.X) 1
camera(took(picture),i,offered(take(us(picture)),they),\X.X) 0
told(them,wait(box(press(\X.X,\X.10_turn(X),way(rest))),\X.X,press(\X.X)),\X.X,\X.X) 0
recommended(picture_quality(canon),\X.X,owned(g2),work(\X.X,\X.10_(X)),\X.X) 1
camera(\X.X,use(include(use(lense(\X.X,flash),ability),10_(\X.10_(X),\X.X,\X.X))),\X.X,line,\X.X) 1
is(looking(get(like(you,\X.adjust(X),\X.X,\X.advanced(X)),flexibility,use(ease,combination,quality(\X.X)))),anyone,recommend(camera),\X.X,\X.X) 1
job_canon(\X.X) 0
yep(\X.X) 0
is(is(toy(\X.X),it,\X.X,camera),\X.X,\X.X) 1
say(\X.X,g3(it(\X.spent(X),cent,i)),i,\X.X,buying(camera,spend(\X.X),i,buy(everything,i,\X.X,\X.X,am(software_engineer),\X.X)),\X.X) 0
gives(\X.X,brilliance(touch),alsmost(\X.X,\X.X,control),is(megapixel(camera,canon(powershot_series,flagship)),powershot_g3(overview)),\X.X) 1
are(expert(no